In [6]:
# Dataconsolidation_
# ChatModded
import requests
import pandas as pd
import json
from datetime import datetime, date, timedelta

Token = 'ziLDJecqBlvTSNGOzOmPPLmBZtXSfBwR'

# Read in zip codes and location names from csv file
referencedata = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')

# Create new column called ZipCode2 by filling missing values in ZipCode column with 0 and converting to int
referencedata['ZipCode2'] = referencedata['ZipCode'].fillna(0).astype(int)

# Get current date and create a list of dates for the past 7 days
today = date.today()
Dateslist = [today - timedelta(days = day) for day in range(7)]
startdate = min(Dateslist)
enddate = max(Dateslist)

# Read in data from csv file
all_data = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentPrecip.csv', index_col=0).drop_duplicates()

# Loop through each row in the reference data
for i, row in referencedata.iterrows():
    name = row['Name']
    zipcode = row['ZipCode']
    zipcode2 = row['ZipCode2']

    # Construct URL to request data from API
    data_url = f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&units=metric&limit=1000&locationid=ZIP:{zipcode}&startdate={startdate}&enddate={enddate}'

    # Send request to API
    r = requests.get(data_url, headers={'token': Token})
    d = json.loads(r.text)

    # If no data is available for the given zip code, try using the zipcode2 value
    if len(d) == 0:
        data_url = f'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=ZIP:{zipcode2}&startdate={startdate}&enddate={enddate}'
        r = requests.get(data_url, headers={'token': Token})
        d = json.loads(r.text)

    # Extract date and precipitation values from API response
    try:
        dates = [item['date'] for item in d['results'] if item['datatype'] == 'PRCP']
        precips = [item['value'] for item in d['results'] if item['datatype'] == 'PRCP']
    except KeyError:
        print('No data available at this Zip Code: ', zipcode2)
    continue

    # Create data frame from date and precipitation values
    df = pd.DataFrame()
    df['Date'] = [datetime.strptime(d, '%Y-%m-%dT%H:%M:%S') for d in dates]
    df['Precip'] = precips
    df['Name'] = name

    # Append data frame to all_data
    all_data = all_data.append(df, ignore_index=True)

# Save all_data to csv file
all_data.to_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentPrecip.csv')

No data available at this Zip Code:  95573
No data available at this Zip Code:  96013
No data available at this Zip Code:  95944
No data available at this Zip Code:  93517
No data available at this Zip Code:  96052
No data available at this Zip Code:  97112
No data available at this Zip Code:  97436
